In [ ]:
# !pip install vllm pandas tqdm joblib
!pip install -q transformers accelerate einops safetensors

In [ ]:
import torch
print("Torch:", torch.__version__)
print("CUDA in torch:", torch.version.cuda)  # ควรขึ้น 12.8
print("Capability:", torch.cuda.get_device_capability(0))  # ควรเป็น (12, 0)
print(torch.cuda.get_device_name(0))

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import textwrap
import os
import json
import time
import requests
import pandas as pd
from itertools import combinations, product
from tqdm.auto import tqdm
from joblib import Parallel, delayed

In [7]:
BLOOM_LEVELS = ["จำ", "เข้าใจ", "นำไปใช้", "วิเคราะห์", "ประเมิน", "สร้างสรรค์"]
LEVELS = ["ง่าย", "ปานกลาง", "ยาก"]
GRADE_LEVELS = ["ม.4", "ม.5", "ม.6"]
TOPICS = ["พีชคณิต"]
QUESTION_TYPES = ["multiple_choice"]

N_JOBS = 8

In [8]:
bloom_combinations = [[b] for b in BLOOM_LEVELS] + [list(pair) for pair in combinations(BLOOM_LEVELS, 2)]
all_combinations = list(product(TOPICS, GRADE_LEVELS, QUESTION_TYPES, LEVELS, bloom_combinations))
print(f"Total combinations: {len(all_combinations)}")

Total combinations: 189


In [9]:
def create_user_prompt(topic, grade_level, question_type, difficulty, bloom_levels, num_problems=1):
    bloom_str = ", ".join(bloom_levels)
    prompt = f"""จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้
1. หัวข้อ: {topic}
2. สำหรับนักเรียน: {grade_level}
3. รูปแบบ: {question_type}
4. ความยาก: {difficulty}
5. bloom level: {bloom_str}
6. จำนวน: {num_problems} ข้อ
7. เพิ่มเติม: โจทย์จำเป็นต้องมีคำตอบ และถ้าโจทย์เป็นแบบ multiple choice (ปรนัย) ต้องมีคำตอบหลอกจำนวน 3 ข้อ (ทั้งหมด หลอก + จริง มี 4 ข้อ) โดยมาจากการคำนวนที่ผิดพลาด"""
    return prompt

In [10]:
model_id = "Qwen/Qwen3-4B-Thinking-2507"
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

# Load tokenizer & model
# trust_remote_code may be needed for some chat templates
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=dtype,
    device_map="auto" if torch.cuda.is_available() else None,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)

model.eval()

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2560, out_features=4096, bias=False)
          (k_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=2560, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (up_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (down_proj): Linear(in_features=9728, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((2560,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [28]:
def call_llm(user_prompt, system_prompt, temperature=0.8, top_p=0.95, max_new_tokens=9216):
    messages = []
    
    messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": user_prompt})

    # Use chat template if available
    if hasattr(tokenizer, "apply_chat_template"):
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(prompt, return_tensors="pt")
    else:
        # Fallback: simple concatenation
        full_prompt = (system_prompt + "\n\n" if system_prompt else "") + user_prompt
        inputs = tokenizer(full_prompt, return_tensors="pt")

    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated_ids = output_ids[0][inputs["input_ids"].shape[1]:]
    text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    return text

In [29]:
def extract_think_and_content(output_str):
    think, content = "", ""
    if "<think>" in output_str and "</think>" in output_str:
        think = output_str.split("<think>")[1].split("</think>")[0].replace("\n", " ").strip()
    if "<questions>" in output_str:
        content = output_str.split("<questions>", 1)[1]
        content = "<questions>" + content
        content = content.replace("\n", " ").strip()
    return think, content


def save_result(think, content, params, result_meta=None, outdir="results"):
    os.makedirs(outdir, exist_ok=True)
    run_id = int(time.time())
    record = {
        "topic": params[0],
        "grade": params[1],
        "qtype": params[2],
        "level": params[3],
        "bloom": params[4],
        "think": think,
        "content": content,
    }
    with open(f"{outdir}/result_{run_id}.json", "w", encoding="utf-8") as f:
        json.dump(record, f, ensure_ascii=False)

In [30]:
# full_prompt = f"{SYSTEM_PROMPT}\n\nUser: {USER_PROMPT}\nAssistant:"

# sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=8192)
# output = llm.generate(full_prompt, sampling_params)

# for output_item in output:
#     prompt = output_item.prompt
#     generated_text = output_item.outputs[0].text
#     wrapped_text = textwrap.fill(generated_text, width=80)
#     print(f"{wrapped_text}")

# result = call_llm(create_user_prompt(*all_combinations[0]))

def _first_text_from_result(result: dict) -> str:
    try:
        out = result.get("output", [])
        if not out:
            return ""
        choices = out[0].get("choices", [])
        if choices:
            msg = choices[0]
            tokens = msg.get("tokens")
            if isinstance(tokens, list):
                return "".join(tokens)
            content = msg.get("message", {}).get("content")
            if isinstance(content, str):
                return content
        return json.dumps(out[0], ensure_ascii=False)
    except Exception:
        return ""


def process_one_combination(params, outdir="results"):
    topic, grade, qtype, level, bloom = params
    user_prompt = create_user_prompt(topic, grade, qtype, level, bloom)
    result_text = call_llm(user_prompt, system_prompt=SYSTEM_PROMPT)
    if not result_text:
        return None

    think, content = extract_think_and_content(result_text)

    # Save as JSON per sample
    meta = None
    save_result(think, content, params=params, result_meta=meta, outdir=outdir)
    return {
        "topic": topic,
        "grade": grade,
        "qtype": qtype,
        "level": level,
        "bloom": bloom,
        "think": think,
        "content": content
    }

In [31]:
# ทดสอบระบบ (full system test 1 รอบ)
try:
    _ = all_combinations[0]
except NameError:
    # ensure combinations are built if this cell runs before
    bloom_combinations = [[b] for b in BLOOM_LEVELS] + [list(pair) for pair in combinations(BLOOM_LEVELS, 2)]
    all_combinations = list(product(TOPICS, GRADE_LEVELS, QUESTION_TYPES, LEVELS, bloom_combinations))

test_result = process_one_combination(all_combinations[0], outdir="test_results")
print("Test result:", {k: (v[:120]+"...") if isinstance(v, str) and len(v) > 120 else v for k, v in (test_result or {}).items()})

Test result: {'topic': 'พีชคณิต', 'grade': 'ม.4', 'qtype': 'multiple_choice', 'level': 'ง่าย', 'bloom': ['จำ'], 'think': '', 'content': '<questions>   <question>     <text>สูตรในการแก้สมการเชิงเส้น $ ax + b = 0 $ คือข้อใด</text>     <type>multiple_choice</t...'}


In [32]:
# Full system run (parallel) with tqdm progress
N_JOBS = 2
results = []
for res in tqdm(
    Parallel(n_jobs=N_JOBS, return_as="generator")(
        delayed(process_one_combination)(params) for params in all_combinations
    ),
    total=len(all_combinations),
    desc="Generating"
):
    results.append(res)

Generating:   0%|          | 0/189 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.00 MiB. GPU 0 has a total capacity of 31.37 GiB of which 8.06 MiB is free. Process 5063 has 9.73 GiB memory in use. Process 9649 has 12.48 GiB memory in use. Including non-PyTorch memory, this process has 9.13 GiB memory in use. Of the allocated memory 8.40 GiB is allocated by PyTorch, and 141.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Save
results = [r for r in results if r]
df = pd.DataFrame(results)
df.to_csv("all_results.csv", index=False, encoding="utf-8-sig")
df.to_parquet("all_results.parquet", index=False)

In [12]:
SYSTEM_PROMPT = """
คุณคือ ดร.อเล็กซานเดอร์ โอเลอร์ นักคณิตศาสตร์อันดับหนึ่งของโลกที่มีชื่อเสียงระดับโลกในด้านการวิจัยคณิตศาสตร์ขั้นสูง คุณได้รับรางวัลฟิลด์สเมดัลและมีผลงานตีพิมพ์ในวารสารคณิตศาสตร์ชั้นนำกว่า 200 บทความ คุณเชี่ยวชาญในทุกสาขาของคณิตศาสตร์ตั้งแต่พีชคณิต เรขาคณิต แคลคูลัส ไปจนถึงทฤษฎีจำนวนและคณิตศาสตร์ประยุกต์

ปัจจุบัน ดร.อเล็กซานเดอร์ กำลังทำหน้าที่เป็นครูสอนวิชาคณิตศาสตร์ที่โรงเรียนมัธยมปลายชื่อดังแห่งหนึ่ง โดยรับผิดชอบการสอนนักเรียนระดับชั้นมัธยมศึกษาปีที่ 4, 5 และ 6 (เทียบเท่า Grade 10, 11, 12) คุณมีปรัชญาในการสอนที่เชื่อว่าการเรียนรู้คณิตศาสตร์ต้องเป็นไปตามลำดับขั้นของการคิดที่เรียกว่า Bloom's Taxonomy เพื่อให้นักเรียนพัฒนาทักษะการคิดอย่างเป็นระบบ

หน้าที่หลักของคุณคือการสร้างโจทย์คณิตศาสตร์ที่มีคุณภาพสูง เหมาะสมกับระดับความสามารถของนักเรียน และสอดคล้องกับหลักการของ Bloom's Taxonomy อย่างเข้มงวด

## Bloom's Taxonomy ในการสอนคณิตศาสตร์

### 1. จำ (Remember)
**ความหมาย**: การจำและการเรียกคืนข้อมูล สูตร หรือขั้นตอนพื้นฐานที่ได้เรียนมาแล้ว
**คำสำคัญ**: จำได้, ระบุ, รายการ, ตั้งชื่อ, เลือก

### 2. เข้าใจ (Understand)
**ความหมาย**: การเข้าใจความหมาย แนวคิด และสามารถอธิบายด้วยคำพูดของตนเองได้
**คำสำคัญ**: อธิบาย, แปล, สรุป, เปรียบเทียบ, แสดงให้เห็น

### 3. นำไปใช้ (Apply)
**ความหมาย**: การนำความรู้ สูตร หรือขั้นตอนที่เรียนมาไปใช้แก้ปัญหาในสถานการณ์ใหม่
**คำสำคัญ**: คำนวณ, แก้, ใช้, แสดง, ดำเนินการ

### 4. วิเคราะห์ (Analyze)
**ความหมาย**: การแยกแยะองค์ประกอบ วิเคราะห์ความสัมพันธ์ และเข้าใจโครงสร้างของปัญหา
**คำสำคัญ**: แยกแยะ, เปรียบเทียบ, ตรวจสอบ, ทดสอบ, วิเคราะห์

### 5. ประเมิน (Evaluate)
**ความหมาย**: การตัดสิน ประเมินค่า ให้เหตุผล และแสดงความคิดเห็นโดยใช้เกณฑ์ที่กำหนด
**คำสำคัญ**: ตัดสิน, ประเมิน, วิจารณ์, แสดงความคิดเห็น, ให้เหตุผล

### 6. สร้างสรรค์ (Create)
**ความหมาย**: การสร้างสรรค์สิ่งใหม่ ออกแบบ วางแผน หรือสร้างโจทย์ปัญหาขึ้นมาเอง
**คำสำคัญ**: สร้าง, ออกแบบ, วางแผน, เสนอ, พัฒนา

## ตัวอย่างโจทย์ตาม Bloom's Taxonomy

### จำ (Remember)
โจทย์: สูตรหาพื้นที่วงกลมคือข้อใด
A) $ A = \pi r^2 $  B) $ A = 2\pi r $  C) $ A = \pi d $  D) $ A = \frac{1}{2}\pi r^2 $

### เข้าใจ (Understand)
โจทย์: อธิบายความหมายของ $ \frac{dy}{dx} = 3x^2 $ ในทางเรขาคณิต

### นำไปใช้ (Apply)
โจทย์: จงหาค่า $ x $ จากสมการ $ 2x + 5 = 13 $

### วิเคราะห์ (Analyze)
โจทย์: เปรียบเทียบพฤติกรรมของฟังก์ชัน $ f(x) = x^2 $ และ $ g(x) = x^3 $ เมื่อ $ x > 1 $

### ประเมิน (Evaluate)
โจทย์: นักเรียนคนหนึ่งอ้างว่า $ \sqrt{a + b} = \sqrt{a} + \sqrt{b} $ เสมอ จงประเมินว่าข้อความนี้ถูกหรือผิด พร้อมให้เหตุผล

### สร้างสรรค์ (Create)
โจทย์: ออกแบบฟังก์ชันกำลังสองที่มีจุดยอดอยู่ที่ $ (2, -3) $ และผ่านจุด $ (0, 1) $

## ตัวอย่างโจทย์แบบผสม 5 โจทย์

### โจทย์ที่ 1 (เข้าใจ + นำไปใช้)
หากฟังก์ชัน $ f(x) = 2x - 3 $ จงหาค่า $ f(5) $ และอธิบายความหมายของผลลัพธ์

### โจทย์ที่ 2 (วิเคราะห์ + ประเมิน)
เปรียบเทียบวิธีแก้สมการ $ x^2 - 5x + 6 = 0 $ ด้วยการแยกตัวประกอบและสูตรกำลังสอง แล้วประเมินว่าวิธีใดมีประสิทธิภาพมากกว่า

### โจทย์ที่ 3 (นำไปใช้ + วิเคราะห์)
ร้านค้าแห่งหนึ่งขายสินค้าในราคา $ 100x - x^2 $ บาท เมื่อขาย $ x $ ชิ้น จงหาจำนวนสินค้าที่ขายได้เงินสูงสุด

### โจทย์ที่ 4 (เข้าใจ + สร้างสรรค์)
จากกราฟ $ y = \sin x $ จงสร้างฟังก์ชันใหม่ที่มีแอมพลิจูดเป็น 3 เท่า และอธิบายการเปลี่ยนแปลง

### โจทย์ที่ 5 (วิเคราะห์ + ประเมิน + สร้างสรรค์)
นักเรียนสองคนแก้โจทย์หาค่าสูงสุดของ $ f(x) = -x^2 + 4x + 1 $ ได้คำตอบต่างกัน คนหนึ่งได้ $ x = 2 $ อีกคนได้ $ x = -2 $ จงวิเคราะห์ว่าใครถูก ประเมินข้อผิดพลาด และสร้างวิธีตรวจสอบคำตอบ

---

## คำสั่งการทำงาน

เมื่อได้รับโจทย์ให้สร้างข้อสอบคณิตศาสตร์ ให้ทำตามขั้นตอนดังนี้:

### ขั้นตอนที่ 1: การคิดและวางแผน
ในแท็ก `<think></think>` ให้ทำการ:
1. **คิดโจทย์เป็นภาษาอังกฤษก่อน** - เพื่อให้ได้แนวคิดที่ชัดเจน
2. **ทดลองหาคำตอบ** - ตรวจสอบว่าโจทย์สามารถแก้ได้จริง
3. **หากแก้ไม่ได้ ให้เปลี่ยนโจทย์** - แล้วทดลองใหม่จนได้โจทย์ที่ดี
4. **เมื่อได้โจทย์ที่สมบูรณ์แล้ว** - สร้างตัวเลือกคำตอบสำหรับ multiple choice (หากต้องการ)
5. **ตรวจสอบความสอดคล้องกับ Bloom's taxonomy** ที่กำหนด

### ขั้นตอนที่ 2: การเขียนผลลัพธ์
หลังจาก `<think></think>` แล้ว ให้เขียนโจทย์ในรูปแบบ XML ตามตัวอย่างนี้:

```xml
<questions>
  <question>
    <text>โจทย์คณิตศาสตร์ที่มี KaTeX formatting เช่น $ 4x + 3 = 2x + 9 $</text>
    <type>multiple_choice</type>
    <options>
      <option>$ ตัวเลือก1 $</option>
      <option>$ ตัวเลือก2 $</option>
      <option>$ ตัวเลือก3 $</option>
      <option>$ ตัวเลือก4 $</option>
    </options>
    <correct_answer>$ คำตอบที่ถูก $</correct_answer>
    <explanation>
      ขั้นตอนที่ 1: อธิบายด้วย KaTeX เช่น $ 4x - 2x + 3 = 9 $
      ขั้นตอนที่ 2: $ 2x + 3 = 9 $
      ขั้นตอนที่ 3: $ 2x = 6 $
      ขั้นตอนที่ 4: $ x = 3 $
    </explanation>
    <score>2</score>
    <difficulty>ง่าย</difficulty>
    <bloom_levels>
      <level>เข้าใจ</level>
      <level>วิเคราะห์</level>
    </bloom_levels>
  </question>

  <question>
    <text>จงหาค่า $ x $ ที่ทำให้ $ 3x - 7 = 2x + 5 $</text>
    <type>short_answer</type>
    <correct_answer>$ x = 12 $</correct_answer>
    <explanation>
      ขั้นตอนที่ 1: นำ $ 2x $ ไปยังข้างซ้าย และ $ 7 $ ไปยังข้างขวา
      $ 3x - 2x = 5 + 7 $
      ขั้นตอนที่ 2: คำนวณ
      $ x = 12 $
    </explanation>
    <score>2</score>
    <difficulty>ปานกลาง</difficulty>
    <bloom_levels>
      <level>เข้าใจ</level>
    </bloom_levels>
  </question>
</questions>
```

## หมายเหตุสำคัญ

### รูปแบบการเขียน:
- **ใช้ KaTeX format** `$ ... $` สำหรับตัวเลข สูตร และนิพจน์คณิตศาสตร์ทั้งหมด
- **ใช้คำไทยสำหรับ Bloom's level**: จำ, เข้าใจ, นำไปใช้, วิเคราะห์, ประเมิน, สร้างสรรค์
- **สามารถมีหลายระดับ Bloom's**: `<bloom_levels><level>เข้าใจ</level><level>นำไปใช้</level></bloom_levels>`

### ระดับความยาก:
- **ง่าย**: โจทย์พื้นฐาน 1-2 ขั้นตอน (คะแนน 1-2)
- **ปานกลาง**: โจทย์ที่ต้องใช้ความคิด 3-4 ขั้นตอน (คะแนน 3-4)
- **ยาก**: โจทย์ซับซ้อน ต้องวิเคราะห์เชิงลึก 5+ ขั้นตอน (คะแนน 5+)

### การให้คะแนน:
- Score ควรสะท้อนระดับความยากและเวลาที่ใช้ในการแก้ปัญหา
- โจทย์ที่ต้องใช้ Bloom's level สูงควรได้คะแนนมากกว่า

คุณพร้อมที่จะสร้างโจทย์คณิตศาสตร์คุณภาพสูงตามที่ได้รับมอบหมายแล้วหรือไม่?"""

USER_PROMPT = """
จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้
1. หัวข้อ: พีชคณิต
2. สำหรับนักเรียน: ม.4
3. รูปแบบ: multiple_choice
4. ความยาก: ยาก
5. bloom level: เข้าใจ, นำไปใช้
6. จำนวน: 1 ข้อ
7. เพิ่มเติม: โจทย์จำเป็นต้องมีคำตอบ และถ้าโจทย์เป็นแบบ multiple choice (ปรนัย) ต้องมีคำตอบหลอกจำนวน 3 ข้อ (ทั้งหมด หลอก + จริง มี 4 ข้อ) โดยมาจากการคำนวนที่ผิดพลาด
"""